In [1]:
import pandas as pd
import re

In [2]:
date = "1904"
carrier = "YD"

inputpath = "G:\\work\\logistica\\info\\input\\cha\\"+date+"cha\\"
inputname = date + "cha_"+carrier+".csv"
outpath = "G:\\work\\logistica\\info\\output\\CHA\\CSV\\"
outname1 = date+"_"+carrier+"_LS_cha"
outname2 = date+"_"+carrier+"_DM_cha"
outname3 = date+"_"+carrier+"_ST_cha"

In [3]:
df = pd.read_csv(inputpath + inputname,sep = None,encoding='utf8')

d:\python\envs\jupyterpy36\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [10]:
df["PARTNER_ACTION"].value_counts()

out        19253
weight      6565
deliver     6248
got         6113
signed      5793
in          5396
arrived     2799
air           46
next          11
Name: PARTNER_ACTION, dtype: int64

In [4]:
df1 = df[df["PARTNER_ACTION"] == "deliver"].reset_index(drop=True) # LS
df2 = df[df["PARTNER_ACTION"] == "deliver"].reset_index(drop=True) # DM
df3 = df[df["PARTNER_ACTION"] == "signed"].reset_index(drop=True) # ST

In [5]:
def get_YD19_lastStations(YD):
    YD_name_list = []
    for i in YD:
        YD_name_dict = {}
        YD_name_dict["YD_lastStation"] = str(re.findall("^(\\w+)\s【",i)).replace("['","").replace("']","")
        YD_name_list.append(YD_name_dict)
    return YD_name_list

In [6]:
def get_YD19_deliveryman(YD):
    YD_name_list = []
    for i in YD:
        YD_name_dict = {}
        YD_name_dict["YD_deliveryman_name"] = str(re.findall("派件员\s(\\w+)\s",i)).replace("['","").replace("']","")
        YD_name_dict["YD_deliveryman_tel"] = str(re.findall("(\\d+)\s正在",i)).replace("['","").replace("']","")
        YD_name_list.append(YD_name_dict)
    return YD_name_list

In [7]:
def get_YD19_status(YD):
    YD_name_list = []
    for i in YD:
        YD_name_dict = {}
        # if "签收人" in YD:
        YD_name_dict["YD_status"] = str(re.findall("快件已被\s(.*)。如有",i)).replace("['","").replace("']","")
        YD_name_list.append(YD_name_dict)
    return YD_name_list

In [8]:
data1 = pd.DataFrame(get_YD19_lastStations(df1["DESCRIPTION"]))
data2 = pd.DataFrame(get_YD19_deliveryman(df2["DESCRIPTION"]))
data3 = pd.DataFrame(get_YD19_status(df3["DESCRIPTION"]))
df_LS = pd.concat([df1,data1],axis=1)
df_DM = pd.concat([df2,data2],axis=1)
df_ST = pd.concat([df3,data3],axis=1)

In [9]:
print("df_LS:" + str(df_LS.shape))
print("df_DM:" + str(df_DM.shape))
print("df_ST:" + str(df_ST.shape))

df_LS:(6248, 13)
df_DM:(6248, 14)
df_ST:(5793, 13)


In [13]:
df_LS.head()

,"﻿""ORDER_ID""",MAIL_NO,ORDER_STATUS,PROV_NAME,CITY_NAME,CHANNEL_ID,LOGISTICS_COMPANY,ROUTE_TIME,ACTION,PARTNER_ACTION,DESCRIPTION,LOGISTICS_CLASS,YD_lastStation
0,600000000001008320190401532716,3102365503981,10112,广东,深圳,NaN,yunda,03-4月 -19 09.38.23.000000000 上午,NaN,deliver,江西新余公司 【新余市】江西新余公司 派件员 胡四豪 13879012746 正在为您派件,0,江西新余公司
1,600000000001008320190401498544,3102365503971,10112,河南,周口,NaN,yunda,03-4月 -19 09.05.25.000000000 上午,NaN,deliver,河南淮阳县公司西城区分部 【周口市】河南淮阳县公司西城区分部 派件员 王艳臣 1589679...,0,河南淮阳县公司西城区分部
2,600000000001008320190401518112,3102365503994,10112,四川,成都,NaN,yunda,03-4月 -19 08.13.07.000000000 上午,NaN,deliver,四川成都金堂县公司 【成都市】四川成都金堂县公司 派件员 陈扬伟 18782016501 正...,0,四川成都金堂县公司
3,600000000001008320190401558569,3102365504104,10112,四川,成都,NaN,yunda,03-4月 -19 08.59.25.000000000 上午,NaN,deliver,河北廊坊公司香河县分部 【廊坊市】河北廊坊公司香河县分部 派件员 王海发 151006828...,0,河北廊坊公司香河县分部
4,600000000001008320190401515143,3102365503972,11202,湖北,十堰,NaN,yunda,03-4月 -19 12.07.49.000000000 下午,NaN,deliver,湖北竹山县公司 【十堰市】湖北竹山县公司 派件员 吴娇 15171363671 正在为您派件,0,湖北竹山县公司
